In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import tensorflow as tf
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-08-20 18:24:11.695 DEBU

In [2]:
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *

### Configuration parameters

In [3]:
NE_TYPES = ['PERSON']
# NE_TYPES = ['PERSON', 'GPE', 'ORG', 'LOC']
N_EXAMPLES = 20
N_TEST_SENTENCES = 100
N_SAMPLES = 1
MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
# MODELS = ['ne_centroid', 'ne_nearest']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}
USE_CV = False

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=True, use_glove=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-20 18:24:22.739 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-08-20 18:24:23.290 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-08-20 18:24:23.547 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel


INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-08-20 18:24:23.553 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-08-20 18:24:23.559 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
2018-08-20 18:24:31.452 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 60: [loading embeddings from `/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-08-20 18:24:31.452 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-08-20 18:24:31.453 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-08-20 18:25:00.246 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 266: loaded (400000, 100) matr

In [6]:
from collections import defaultdict
f1_scores = {}
th_list = {}
for ne_type in NE_TYPES:
    print('NE type: {}'.format(ne_type))
    dataset = filter_dataset_by_ne_types(dataset_orig, ne_type, preserveBIO=True)
    f1_scores[ne_type] = {}
    th_list[ne_type] = {'ne_centroid':[], 'ne_nearest':[]}
    for model in MODELS:
        f1_scores[ne_type][model] = {'values':[], 'values_chunk':[]}
#     f1_scores[ne_type]['avg'] = {'values':[]}'ne_centroid', 'ne_nearest'
    for idx_sample in range(N_SAMPLES):
        print('Sample {}'.format(idx_sample+1))
        tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
        tags_train = removeBIOFromTags(tags_train)
        tokens_test,tags_test = get_data_sample(dataset['test'], N_TEST_SENTENCES)
        tags_test_bio = tags_test
        tags_test_bio_flat = flatten_list(tags_test_bio)
        tags_test = removeBIOFromTags(tags_test)
        y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
        ner_bin_clf = FewshotNerBinaryClassifier(embedder)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        if USE_CV:
            cv_res = select_sim_thresholds_with_cv(tokens_train,tags_train, embedder)
        pred_avg = np.zeros(y_test.size)
        n_models_avg = 0
        for model in MODELS:
            if model in ['ne_centroid', 'ne_nearest'] and USE_CV:
                pred= pred_class_labels_bin(flat_sim_one_type(results[model]['sim'], sim_type='cosine'), cv_res[model]['threshold'])
                th_list[ne_type][model].append(cv_res[model]['threshold'])
            else:
                pred = results[model]['pred']
            pred_tags_bio = predToTags(pred, accountBIO=True)
            if model in ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']:   
                pred_avg += pred
                n_models_avg += 1
            f1 = f1_score(y_test, pred)
#             print(tags_test_bio)
#             print(pred_tags_bio)
            f1_chunk = precision_recall_f1_chunks(tags_test_bio_flat, pred_tags_bio, print_results=False)['__total__']['f1']
            f1_scores[ne_type][model]['values'].append(f1)
            f1_scores[ne_type][model]['values_chunk'].append(f1_chunk)
        pred_avg /= n_models_avg
#         pred_avg = np.array(map(lambda x: 1 if x >= 0.5 else 0, pred_avg))
#         f1 = f1_score(y_test, pred_avg)
#         f1_scores[ne_type]['avg']['values'].append(f1)
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 scores: {}'.format(cur_f1_scores_dict))
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values_chunk'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 chunk scores: {}'.format(cur_f1_scores_dict))                                          
    for model in MODELS:
        f1_scores[ne_type][model]['mean'] = np.mean(f1_scores[ne_type][model]['values'])
        f1_scores[ne_type][model]['std'] = np.std(f1_scores[ne_type][model]['values'])
        print('{} F1: mean = {} std = {}'.format(model, f1_scores[ne_type][model]['mean'], f1_scores[ne_type][model]['std']))
        f1_scores[ne_type][model]['mean_chunk'] = np.mean(f1_scores[ne_type][model]['values_chunk'])
        f1_scores[ne_type][model]['std_chunk'] = np.std(f1_scores[ne_type][model]['values_chunk'])
        if model in ['ne_centroid', 'ne_nearest'] and USE_CV:
            th_list[ne_type][model] = np.mean(th_list[ne_type][model])

NE type: PERSON
Num of train sentences: 12195
Num of valid sentences: 1553
Num of test sentences: 1573
Sample 1
Train
NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.7839999999999999, 'ne_nearest': 0.4308681672025723, 'weighted_kNN': 0.8798370672097761, 'centroid_kNN': 0.8030303030303029, 'svm': 0.8720000000000001}
F1 chunk scores: {'ne_centroid': 56.59163987138263, 'ne_nearest': 21.105527638190956, 'weighted_kNN': 72.92418772563177, 'centroid_kNN': 61.72839506172839, 'svm': 73.75886524822694}
ne_centroid F1: mean = 0.7839999999999999 std = 0.0
ne_nearest F1: mean = 0.4308681672025723 std = 0.0
weighted_kNN F1: mean = 0.8798370672097761 std = 0.0
centroid_kNN F1: mean = 0.8030303030303029 std = 0.0
svm F1: mean = 0.8720000000000001 std = 0.0


### Print results

In [7]:
print('{:<8s}\t{:<8s}\t{:<8s}'.format('MODEL', 'MEAN', 'STD', 'MEAN_CHUNK', 'STD_CHUNK'))
for ne_type in NE_TYPES:
    print('---------------{}---------------'.format(ne_type))
    for model in MODELS:
        print('{:<8s}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}'.format(model, f1_scores[ne_type][model]['mean']*100, f1_scores[ne_type][model]['std']*100,
                                                f1_scores[ne_type][model]['mean_chunk']*100, f1_scores[ne_type][model]['std_chunk']*100))

MODEL   	MEAN    	STD     
---------------PERSON---------------
ne_centroid	78.40   	0.00    	5659.16 	0.00    
ne_nearest	43.09   	0.00    	2110.55 	0.00    
weighted_kNN	87.98   	0.00    	7292.42 	0.00    
centroid_kNN	80.30   	0.00    	6172.84 	0.00    
svm     	87.20   	0.00    	7375.89 	0.00    


In [8]:
if USE_CV:
    print('Thresholds chosen with cross-validation:')
    print(th_list)